In [1]:
#需要先修改excel，比如 excel里面龚婕(龚洁)，而EEG数据为AV-龚洁-20220627，此时请将excel中的龚婕(龚洁)改为龚洁
#否则排序和匹配会出问题
#还要去掉AV-周牧涵-20220518 这个有误的数据
#将第二批刺激参数表的龙元芳 20220516的标签改为-，因为并没有这个数据

In [2]:
from mne.io import concatenate_raws, read_raw_edf
import mne
import glob
import numpy as np
import xlrd
import pandas as pd

In [3]:
parameter_path=['../SE_parameter/刺激参数（第一批次）.xlsx',
                '../SE_parameter/刺激参数（第三批次）.xlsx',
                '../SE_parameter/刺激参数（第三批次）.xlsx',
                '../SE_parameter/刺激参数（第四批次）.xlsx']

In [4]:
# 性别和年龄暂时不用，且第二批数据没有这两项
df=pd.DataFrame(columns=['姓名','治疗时间','刺激方案','标签'])
df

,姓名,治疗时间,刺激方案,标签


In [5]:
for path in parameter_path:
    df = pd.concat([df,pd.read_excel(path,engine='openpyxl')],ignore_index=True,join='inner')
df

,姓名,治疗时间,刺激方案,标签
0,曾刚,20220216,2,1
1,曾刚,20220218,3,0
2,曾刚,20220221,4,1
3,曾刚,20220223,1,1
4,杜娇,20220316,4,1
...,...,...,...,...
379,张海峰,20220819,4,0
380,张会,20220713,1,1
381,张会,20220718,2,0
382,张会,20220720,3,1


In [6]:
# path = "../SE_parameter/刺激参数（第四批次）.xlsx"
# df = pd.read_excel(path,engine='openpyxl')
# #df= df.astype({'完成时间':'object'})
# #对df进行排序，为了和glob.glob的顺序相匹配
# df

In [7]:
df=df.sort_values(by=["姓名","治疗时间"])
df

,姓名,治疗时间,刺激方案,标签
371,严聪慧,20220803,4,0
370,严聪慧,20220805,3,0
368,严聪慧,20220808,1,1
369,严聪慧,20220810,2,1
164,习俊豪,20220525,3,1
...,...,...,...,...
209,龚洁,20220622,4,1
114,龚洁,20220624,1,1
210,龚洁,20220624,1,1
115,龚洁,20220627,3,1


In [8]:
# 去掉差的数据
df.drop(df.loc[df['标签']=='-'].index, inplace=True)
df.drop(df.loc[df['标签'].isnull()].index, inplace=True)
df=df.reset_index()
df=df.drop(['index'],axis=1)
df

,姓名,治疗时间,刺激方案,标签
0,严聪慧,20220803,4,0
1,严聪慧,20220805,3,0
2,严聪慧,20220808,1,1
3,严聪慧,20220810,2,1
4,习俊豪,20220525,3,1
...,...,...,...,...
378,龚洁,20220622,4,1
379,龚洁,20220624,1,1
380,龚洁,20220624,1,1
381,龚洁,20220627,3,1


In [9]:
# 将标签列数据类型转换为int
df["标签"] = df["标签"].astype(int)  
df

,姓名,治疗时间,刺激方案,标签
0,严聪慧,20220803,4,0
1,严聪慧,20220805,3,0
2,严聪慧,20220808,1,1
3,严聪慧,20220810,2,1
4,习俊豪,20220525,3,1
...,...,...,...,...
378,龚洁,20220622,4,1
379,龚洁,20220624,1,1
380,龚洁,20220624,1,1
381,龚洁,20220627,3,1


In [10]:
# rest_path = r"../patient_data/静息脑电（第一批次）/"
therapy_path=[r"../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考/",
              r"../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考/",
              r"../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考/"]

# rest_subdirectory = sorted(glob.glob(rest_path+"*"))
# therapy_subdirectory = sorted(glob.glob(therapy_path+"*"))
therapy_file_path=[]
for path in therapy_path:
    therapy_file_path = therapy_file_path+sorted(glob.glob(path+"*"))
#这里必须整体重新排序
therapy_file_path=sorted(therapy_file_path,key=lambda x:x.replace('\\','/').split('/')[-1].split('-')[1])
len(therapy_file_path)

285

In [11]:
therapy_file_path

['../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220525.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220527.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220530.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220606.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-2022083

In [12]:
path_parameter_dict={}
path_result_dict={}

In [13]:
df

,姓名,治疗时间,刺激方案,标签
0,严聪慧,20220803,4,0
1,严聪慧,20220805,3,0
2,严聪慧,20220808,1,1
3,严聪慧,20220810,2,1
4,习俊豪,20220525,3,1
...,...,...,...,...
378,龚洁,20220622,4,1
379,龚洁,20220624,1,1
380,龚洁,20220624,1,1
381,龚洁,20220627,3,1


In [14]:
therapy_file_path

['../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220525.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220527.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220530.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220606.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-2022083

In [15]:
#构建一个 {路径:ECT参数} 的字典，方便打乱顺序后，仍能保持一一对应
for i in range(len(therapy_file_path)):
    path_parameter_dict[therapy_file_path[i]]=df.iloc[i,2]
    path_result_dict[therapy_file_path[i]]=df.iloc[i,3]

In [16]:
path_parameter_dict

{'../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf': 3,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf': 2,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220525.edf': 3,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220527.edf': 3,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220530.edf': 2,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220606.edf': 2,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf': 2,
 '../patie

In [17]:
path_result_dict

{'../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf': 0,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf': 0,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf': 1,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220525.edf': 1,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220527.edf': 1,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220530.edf': 1,
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220606.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf': 1,
 '../patie

In [18]:
#观察文件名，发现所有文件名均为“-人名-”，所以取人名前后对照，如果不同则表示是不同的人
#思路：首先将所有人名取出，然后随机打乱顺序，前20个人作为训练数据，后4个人作为测试数据
therapy_file_path[0].replace('\\','/').split('/')[-1].split('-')[1]

'严聪慧'

In [19]:
#获得名单
def getNameList(file_path):
    name_list=[]
    name_path_dict={}
    temp=[]
    cur_name='anonymous'
    for i in file_path:
        next_name=i.replace('\\','/').split('/')[-1].split('-')[1]
        if cur_name!=next_name:
            name_path_dict[cur_name]=temp
            cur_name=next_name
            name_list.append(cur_name)
            temp=[]             
        temp.append(i)      
    #也要把最后一个人写入词典
    name_path_dict[cur_name]=temp
    return name_list,name_path_dict

In [20]:
name_list,name_path_dict=getNameList(therapy_file_path)

In [21]:
name_path_dict

{'anonymous': [],
 '严聪慧': ['../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf'],
 '习俊豪': ['../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220525.edf',
  '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220527.edf',
  '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220530.edf',
  '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220606.edf'],
 '付宇欣': ['../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf'],
 '何晓丽': ['../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf',
  '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-202208

In [22]:
#打乱名单，体现随机性
np.random.seed(42)
np.random.shuffle(name_list)
len(name_list)

72

In [23]:
#此处设定有多少患者用于训练集(人数)
train_patient_num=60
total_patient_num=len(name_list)
train_therapy_file_path=[]
test_therapy_file_path=[]
for i in range(train_patient_num):
    for j in name_path_dict[name_list[i]]:
        train_therapy_file_path.append(j)
for i in range(train_patient_num,total_patient_num):
    for j in name_path_dict[name_list[i]]:
        test_therapy_file_path.append(j)

In [24]:
#这里必须要排序
train_therapy_file_path.sort(key=lambda x:x.replace('\\','/').split('/')[-1].split('-')[1])
test_therapy_file_path.sort(key=lambda x:x.replace('\\','/').split('/')[-1].split('-')[1])

In [25]:
train_therapy_file_path

['../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220831.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220511.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220516.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220518.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220520.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-20220411.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-20220413.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-20220415.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-2022042

In [26]:
# raw=read_raw_edf(train_therapy_file_path[0],preload=False)
# print(raw.info.ch_names)

In [27]:
df

,姓名,治疗时间,刺激方案,标签
0,严聪慧,20220803,4,0
1,严聪慧,20220805,3,0
2,严聪慧,20220808,1,1
3,严聪慧,20220810,2,1
4,习俊豪,20220525,3,1
...,...,...,...,...
378,龚洁,20220622,4,1
379,龚洁,20220624,1,1
380,龚洁,20220624,1,1
381,龚洁,20220627,3,1


In [28]:
#统计未发作和发作的样本数
trigger_num=0
non_trigger_num=0
for i in range(df.shape[0]):
    # 找不到未发作，即发作
    if df.iloc[i, 3]==1:
        trigger_num+=1
    else:
        non_trigger_num+=1
print(trigger_num/non_trigger_num)

multiple=trigger_num/non_trigger_num

7.148936170212766


In [29]:
trigger_num

336

In [30]:
non_trigger_num

47

In [28]:
multiple = round(multiple)
multiple

9

In [29]:
non_trigger_num

29

In [30]:
trigger_num+non_trigger_num*multiple

519

In [31]:
#抖动
def jitter(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            #抖动因子
            jitter_factor=np.random.randint(1,11)
            #抖动方向
            jitter_direction=np.random.randint(1,3)
            #变化在 *0.9 至 *1.1内
            if jitter_direction==1:
                data[i][j]= data[i][j]*(1+jitter_factor/100)
            else:
                data[i][j]= data[i][j]*(1-jitter_factor/100)

In [32]:
#翻转
def flip(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]=-data[i][j]

In [33]:
#随机通道翻转
def random_flip(data):
    flip_channel=np.random.randint(0,15)
    for i in range (data.shape[1]):
        data[flip_channel][i]=-data[flip_channel][i]

In [34]:
#扩大/压缩
def scale_up(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*1.1
def scale_down(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*0.9

In [35]:
# #排列
# def permutation(data,p_strategy):
#     for i in range(0,16,2):
#         #把2s片段分为4组,假设序号为1,2,3,4
#         p1_start=int(i*1024)
#         p1_end=int((i+0.5)*1024)
#         p2_start=int((i+0.5)*1024)
#         p2_end=int((i+1)*1024)
#         p3_start=int((i+1)*1024)
#         p3_end=int((i+1.5)*1024)
#         p4_start=int((i+1.5)*1024)
#         p4_end=int((i+2)*1024)
#         for j in range(15):
#             #排列方案为1时， 1,2,3,4-->2,3,4,1
#             if p_strategy==1: 
#                 data_temp=[]
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data=np.array(data)
#                 data[j,p1_start:p4_end]=data_temp
#             #排列方案为2时， 1,2,3,4-->3,4,1,2
#             elif p_strategy==2:
#                 data_temp=[]
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data[j,p1_start:p4_end]=data_temp
#             #排列方案为3时， 1,2,3,4-->4,1,2,3
#             elif p_strategy==3:
#                 data_temp=[]
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data[j,p1_start:p4_end]=data_temp

In [36]:
#检测EEG文件是否有flag标记,返回true/false
def judgeflag(file_path,flag):
    raw = read_raw_edf(file_path,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    if flag in event_dict:
        return True
    else:
        return False

In [37]:
# raw=read_raw_edf(train_therapy_file_path[0],preload=False)

In [38]:
# events_from_annot, event_dict = mne.events_from_annotations(raw)

In [39]:
# event_dict

In [40]:
# events_from_annot

In [41]:
# data,times=raw[:,0:1024*16]
# raw2=mne.io.RawArray(data,raw.info)

In [42]:
# 数据采样率1024Hz,文件路径；采集时间；是否为测试集标志;数据增强策略;未发作的第几次随机通道翻转
def get_se_epochs(path,select_time,is_test,DA_strategy,random_flip_no):
    #选择15个通道
    chs1 = ['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV',
       'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV','EEG Cz-AV']
#     chs2 =['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1','EEG F4-A2','EEG C3-A1','EEG C4-A2','EEG P3-A1','EEG P4-A2','EEG O1-A1',
#            'EEG O2-A2','EEG T3-A1','EEG T4-A2','EEG T5-A1','EEG T6-A2','EEG Cz-AAV']
    # select_time 采集的时间
    raw=read_raw_edf(path,preload=False)
    print(raw.info.ch_names)
    #第一批数据采样为1024Hz，第二批变成了512Hz
#     if chs1[0] in raw.info.ch_names:
#         chs=chs1  
#         freq=1024
#     else:
#         chs=chs2
#         freq=512
    chs=chs1
    freq=512
    raw=raw.pick_channels(chs)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    # 找到SE，麻醉标记,save函数的参数单位是s，所以要除以1024
    start_time=events_from_annot[event_dict['SE']-2][0]
    end_time=start_time+select_time*freq
    print(start_time,end_time)
    #选择所有的15通道
    data,times = raw[:,start_time:end_time]
    print(times)
    #不做变化
    if (DA_strategy==0): 
        path_name_suffix='(original)'
    #抖动
    elif (DA_strategy==1):
        jitter(data)
        path_name_suffix='(jitter)'
    #翻转
    elif (DA_strategy==2):
        flip(data)
        path_name_suffix='(flip)'
    #扩大
    elif (DA_strategy==3):
        scale_up(data)
        path_name_suffix='(scale_up)'  
    #缩小
    elif (DA_strategy==4):
        scale_down(data)
        path_name_suffix='(scale_down)'  
    #排列1
    elif (DA_strategy==5):
        permutation(data,1)
        path_name_suffix='(permutation_1)'  
    #排列2
    elif (DA_strategy==6):
        permutation(data,2)
        path_name_suffix='(permutation_2)'  
    #排列3
    elif (DA_strategy==7):
        permutation(data,3)
        path_name_suffix='(permutation_3)'  
    #随机通道翻转
    elif (DA_strategy==8):
        random_flip(data)
        path_name_suffix='random_flip_'+str(random_flip_no)
    raw_temp=mne.io.RawArray(data,raw.info)
    path_name=path.split('\\')[1:]
    path_name="".join(path_name)
    path_name=path_name.split('.')[0]
    #在这里将训练集和测试集分开,需要新建SE_train和SE_test两个文件夹
    if (is_test==False):
        path_name=r"../SE_train/" + path_name +path_name_suffix+".fif"
    else:   
        path_name=r"../SE_test/" + path_name + ".fif"     
    path_name=path_name.replace('/','\\')
    print(path_name)
    raw_temp.save(path_name,overwrite=True)

In [43]:
file_without_se = []
df_parameter_train=pd.DataFrame(columns=['parameter','result'])
df_parameter_test=pd.DataFrame(columns=['parameter','result'])

In [44]:
train_therapy_file_path

['../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220831.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220511.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220516.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220518.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-何金蓓-20220520.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-20220411.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-20220413.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-20220415.edf',
 '../patient_data/治疗脑电-导出/第二批次治疗脑电/平均参考\\AV-余琛宇-2022042

In [45]:
#训练集SE切片长度
train_SE_len=16
#测试集SE切片长度
test_SE_len=16
#选择offset是多少的倍数
# offset_multiple=2

In [46]:
#生成SE_train
for file_path in train_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #初始化random_flip_no，random_flip_no的作用是方便命名未发作文件，防止发生同名文件覆盖
        random_flip_no=0
        #如果发作，则只截取train_SE_len长度
        #为False时表示当前数据送入训练集
        #通过8种策略数据增强
        for DA_strategy in range(5):
            get_se_epochs(file_path,train_SE_len,False,DA_strategy,0)
            #添加参数到df_parameter_train的末尾
            df_parameter_train.loc[len(df_parameter_train.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
            #如果未发作，则随机通道翻转增加未发作样本数量
            if path_result_dict[file_path]==0:
                for j in range(1,multiple):
                    #修改random_flip_no
                    random_flip_no+=1
                    #DA_strategy固定为8，即通过随机翻转达到0-1平衡
                    get_se_epochs(file_path,train_SE_len,False,8,random_flip_no) 
                    df_parameter_train.loc[len(df_parameter_train.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
    else:
        file_without_se.append(file_path)
#生成SE_test,不考虑对未发作数据的重叠采样
for file_path in test_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #取test_SE_len片段
        #DA_strategy=0,即不做变化
        get_se_epochs(file_path,test_SE_len,True,0,0) 
        df_parameter_test.loc[len(df_parameter_test.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
    else:
        file_without_se.append(file_path)

Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1774 9966
[ 3.46484375  3.46679688  3.46875    ... 19.45898438 19.4609375
 19.46289062]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 .

EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1774 9966
[ 3.46484375  3.46679688  3.46875    ... 19.45898438 19.4609375
 19.46289062]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_train\AV-严聪慧-20220803random_flip_8.fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803random_flip_8.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803random_flip_8.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file det

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(original).fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803random_flip_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarni

Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_train\AV-严聪慧-20220803(jitter).fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(jitter).fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(jitter).fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1774 9966
[ 3.46484375  3.46679688  3.46875    ... 19.45898438 19.4609375
 19.46289062]
Creating RawArray with floa

[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1774 9966
[ 3.46484375  3.46679688  3.46875    ... 19.45898438 19.4609375
 19.46289062]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_train\AV-严聪慧-20220803(flip).fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(flip).fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(flip).fif
[done]
Extracting EDF parameters 

...

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-罗文彤-20220727.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-邓敏-20220606.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupy

    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_test\AV-邓敏-20220613.fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-邓敏-20220613.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-邓敏-20220613.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第三批次治疗脑电\平均参考\AV-邓敏-20220615.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'start']
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第三批次治疗脑电\平均参考\AV-邓敏-20220615.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations des

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-邓敏-20220613.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-邓敏-20220615.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyt

Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第二批次治疗脑电\平均参考\AV-魏平-20220513.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1455 9647
[ 2.84179688  2.84375     2.84570312 ... 18.8359375  18.83789062
 18.83984375]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_test\AV-魏平-20220513.fif
Writing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-魏平-20220513.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-魏平-20220513.fif
[done]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9124\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-魏平-20220513.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)


In [47]:
file_without_se

['../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-刘汉悦-20220613.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-施飞腾-20220606.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-施飞腾-20220608.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-朱超-20220617.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-朱超-20220620.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-李艳雪-20220523.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-樊茂-20220608.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-邹立-20220525.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-龚洁-20220622.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220527.edf']

In [48]:
df_parameter_train

,parameter,result
0,4,0
1,4,0
2,4,0
3,4,0
4,4,0
...,...,...
2055,1,1
2056,1,1
2057,1,1
2058,1,1


In [49]:
test_therapy_file_path

['../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220525.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220527.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220530.edf',
 '../patient_data/治疗脑电-导出/第三批次治疗脑电/平均参考\\AV-习俊豪-20220606.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-吴妍-20220817.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-吴妍-20220819.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-吴妍-20220822.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-吴妍-20220824.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-张会-20220713.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-张会-20220715.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-张会-20220718.edf',
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-张会-20220720.edf',


In [50]:
df_parameter_test

,parameter,result
0,3,1
1,2,1
2,2,1
3,1,1
4,1,1
5,4,1
6,4,1
7,2,1
8,3,1
9,4,1


In [51]:
train_output_path=("../SE_parameter/train_matched_stimulate_parameter.xlsx")
test_output_path=("../SE_parameter/test_matched_stimulate_parameter.xlsx")

In [52]:
df_parameter_train.to_excel(train_output_path,sheet_name="Sheet1")

In [53]:
df_parameter_test.to_excel(test_output_path,sheet_name="Sheet1")